<a href="https://colab.research.google.com/github/bopeng-sue/Optimal-Biofluid-Matrices-for-Human-Exposome-Biomonitoring/blob/main/hyperparameter_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, MACCSkeys, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    balanced_accuracy_score,
    roc_auc_score,
    classification_report,
    roc_curve,
    precision_recall_curve,
    auc
)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from tqdm import tqdm
from pathlib import Path
from imblearn.over_sampling import SMOTE
import warnings
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

warnings.filterwarnings('ignore')
RDLogger.DisableLog('rdApp.*')

In [ ]:
df = pd.read_excel('dataset_1211.xlsx')
df.head(2)

,Unnamed: 0,Biomonitor Compound PubChem CID,Blood study numbers,Urine study numbers,Tendency for the selection of blood samples,Biospecimen,CAS Number,Name,SMILES,hhlb(hours),...,JGI10,JGT,VE1_D,VE3_D,VR1_D,VR2_D,SRW5,AMW,WTPT-3,XLogP
0,0,70,0,2,0.0,Urine,816-66-0,4-Methyl-2-oxovaleric acid,CC(C)CC(=O)C(=O)O,0.61,...,0.0,0.601944,0.046505,-2.761367,35.180772,3.908975,0.0,6.845421,7.158910,0.904
1,1,89,0,1,0.0,Urine,484-78-6,Hydroxykynurenine,C1=CC(=C(C(=C1)O)N)C(=O)CC(C(=O)O)N,0.43,...,0.0,0.556317,0.130944,-3.252780,192.246721,12.015420,0.0,8.002847,14.772289,-2.464


# feature function

In [ ]:
def padel_descriptor(df: pd.DataFrame) -> np.ndarray:
    """
    Extracts Padel descriptors from a given DataFrame starting from column index 16
    to the end. It replaces NaN values with 0 and infinite values with 1e10.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing feature columns.

    Returns
    -------
    np.ndarray
        A NumPy array of transformed feature values.
    """
    # Slice columns from index 16 to the end
    X = df.iloc[:, 16:].values

    # Replace NaNs with 0
    X = np.nan_to_num(X, nan=0.0)

    # Replace infinities with a large finite value
    X[np.isinf(X)] = 1e10

    return X

In [ ]:
def maccs_descriptor(smiles_list):
    """
    Generates MACCS fingerprints for a list of SMILES strings and returns them as a DataFrame.

    Parameters
    ----------
    smiles_list : list of str
        A list of SMILES strings for which MACCS keys will be generated.

    Returns
    -------
    pd.DataFrame
        A DataFrame of MACCS bit values (0 or 1) with column names 'MACCS_1' through 'MACCS_167'.
    """
    # Generate MACCS fingerprints and convert them to a list of bits
    maccs_fingerprints = [
        list(MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles)).ToBitString())
        for smiles in smiles_list
    ]

    # Convert the list of bits to a DataFrame
    # MACCSkeys are 167 bits long, hence 167 columns
    X = pd.DataFrame(maccs_fingerprints, columns=[f'MACCS_{i}' for i in range(1, 168)])

    # Convert strings to integers (0 and 1)
    X = X.astype(int)

    return X

In [ ]:
def ecfp_descriptor(smiles_list, radius=2, n_bits=1024):
    """
    Generates ECFP (Extended-Connectivity Fingerprints) for a list of SMILES strings
    and returns them as a NumPy array.

    Parameters
    ----------
    smiles_list : list of str
        A list of SMILES strings for which ECFP fingerprints will be generated.
    radius : int, optional (default=2)
        The fingerprint radius to use. For ECFP4, radius=2 is standard.
    n_bits : int, optional (default=1024)
        Length of the bit vector.

    Returns
    -------
    np.ndarray
        A 2D NumPy array where each row corresponds to the ECFP bit vector of the input SMILES.
    """
    def calculate_ecfp(smiles, radius=radius, n_bits=n_bits):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=n_bits, useFeatures=False)
            return np.array(fp)
        else:
            return np.zeros(n_bits, dtype=int)

    # Apply ECFP calculation to the input list of SMILES
    ecfp_fingerprints = np.array([calculate_ecfp(s) for s in smiles_list])
    return ecfp_fingerprints

In [ ]:
def fcfp_descriptor(smiles_list, radius=2, n_bits=1024):
    """
    Generates FCFP (Feature-based Circular Fingerprints) for a list of SMILES strings
    and returns them as a NumPy array.

    Parameters
    ----------
    smiles_list : list of str
        A list of SMILES strings for which FCFP fingerprints will be generated.
    radius : int, optional (default=2)
        The fingerprint radius to use.
    n_bits : int, optional (default=1024)
        Length of the bit vector.

    Returns
    -------
    np.ndarray
        A 2D NumPy array where each row corresponds to the FCFP bit vector of the input SMILES.
    """
    def calculate_fcfp(smiles, radius=radius, n_bits=n_bits):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=n_bits, useFeatures=True)
            return np.array(fp)
        else:
            return np.zeros(n_bits, dtype=int)

    # Apply FCFP calculation to the input list of SMILES
    fcfp_fingerprints = np.array([calculate_fcfp(s) for s in smiles_list])
    return fcfp_fingerprints

In [ ]:
def rdkit_descriptors_to_X(df: pd.DataFrame, smiles_col='SMILES', correlation_threshold=0.95) -> np.ndarray:
    """
    Compute RDKit molecular descriptors for each SMILES in the given DataFrame,
    remove descriptors with zero variance and highly correlated descriptors,
    and return the final feature matrix X.

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame containing at least a 'SMILES' column.
    smiles_col : str, optional
        The name of the column containing SMILES strings.
    correlation_threshold : float, optional
        Threshold above which descriptors are considered highly correlated and removed.

    Returns
    -------
    np.ndarray
        A 2D NumPy array containing the filtered RDKit descriptors.
    """
    # Convert SMILES to RDKit Mol objects
    mol_list = [Chem.MolFromSmiles(s) for s in df[smiles_col]]

    # Define the descriptor names
    descriptor_names = [desc_name[0] for desc_name in Descriptors._descList]

    # Create a descriptor calculator
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

    # Calculate descriptors for each molecule, if mol is None use NaNs
    descriptors_list = [
        calculator.CalcDescriptors(mol) if mol is not None else [np.nan]*len(descriptor_names)
        for mol in mol_list
    ]

    # Convert the list of descriptors to a DataFrame
    descriptors_df = pd.DataFrame(descriptors_list, columns=descriptor_names)

    # Replace NaNs with 0
    descriptors_df = descriptors_df.fillna(0)

    # Step 1: Remove descriptors with zero variance
    variance = descriptors_df.var()
    zero_variance_columns = variance[variance == 0].index
    descriptors_df = descriptors_df.drop(columns=zero_variance_columns)

    # Step 2: Remove descriptors with correlation exceeding the threshold
    corr = descriptors_df.corr().abs()
    to_drop = []

    for i, col1 in enumerate(corr.columns):
        for col2 in corr.columns[i+1:]:
            if corr.loc[col1, col2] > correlation_threshold:
                to_drop.append(col2)

    # Remove duplicates if any
    to_drop = list(set(to_drop))

    # Drop the highly correlated descriptors
    descriptors_df = descriptors_df.drop(columns=to_drop, errors='ignore')

    # Convert the final descriptors DataFrame to a NumPy array
    X = descriptors_df.values

    return X

# New Section

In [ ]:
def compute_descriptors(df: pd.DataFrame, descriptor_type: str, smiles_col: str='SMILES') -> np.ndarray:
    """
    Computes the specified molecular descriptor for the given DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing at least the 'SMILES' column.
    descriptor_type : str
        Type of descriptor to compute. One of ['padel', 'maccs', 'ecfp', 'fcfp', 'rdkit'].
    smiles_col : str, optional
        Name of the column containing SMILES strings.

    Returns
    -------
    np.ndarray
        Feature matrix as a NumPy array.
    """
    if descriptor_type.lower() == 'padel':
        X = padel_descriptor(df)
    elif descriptor_type.lower() == 'maccs':
        X = maccs_descriptor(df[smiles_col].tolist()).values
    elif descriptor_type.lower() == 'ecfp':
        X = ecfp_descriptor(df[smiles_col].tolist())
    elif descriptor_type.lower() == 'fcfp':
        X = fcfp_descriptor(df[smiles_col].tolist())
    elif descriptor_type.lower() == 'rdkit':
        X = rdkit_descriptors_to_X(df, smiles_col=smiles_col, correlation_threshold=0.95)
    else:
        raise ValueError("Invalid descriptor_method. Choose from 'padel', 'maccs', 'fcfp', 'ecfp', 'rdkit'.")
    return X

In [ ]:
def run_ml_pipeline(
    X: np.ndarray,
    y: np.ndarray,
    descriptor_type: str,
    model_output_path: str='models/',
    max_evals: int=10000,
    random_state: int=42
) -> dict:
    """
    Runs the machine learning pipeline for a given descriptor type using CatBoost,
    trains the best model, evaluates it, plots ROC and PRC curves, saves the model,
    and returns performance metrics.

    Parameters
    ----------
    X : np.ndarray
        Feature matrix.
    y : np.ndarray
        Target vector.
    descriptor_type : str
        Descriptor type being processed.
    model_output_path : str, optional
        Directory path to save the models and plots. Defaults to 'models/'.
    max_evals : int, optional
        Number of evaluations for hyperparameter optimization. Defaults to 10000.
    random_state : int, optional
        Random state for reproducibility. Defaults to 42.

    Returns
    -------
    dict
        A dictionary containing performance metrics.
    """
    # Create output directory if it doesn't exist
    Path(model_output_path).mkdir(parents=True, exist_ok=True)

    # ---------------------------
    # 1. Data Splitting
    # ---------------------------

    # Initialize the first split: train + temp (validation + test)
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=random_state)
    for train_index, temp_index in sss1.split(X, y):
        X_train, X_temp = X[train_index], X[temp_index]
        y_train, y_temp = y[train_index], y[temp_index]

    # Initialize the second split: validation + test from temp
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=2/3, random_state=random_state)  # 1/3 validation, 2/3 test
    for val_index, test_index in sss2.split(X_temp, y_temp):
        X_val, X_test = X_temp[val_index], X_temp[test_index]
        y_val, y_test = y_temp[val_index], y_temp[test_index]

    print(f"Dataset sizes for {descriptor_type}:")
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Validation set: X_val: {X_val.shape}, y_val: {y_val.shape}")
    print(f"Test set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    # ---------------------------
    # 2. Apply SMOTE
    # ---------------------------

    smote = SMOTE(random_state=random_state)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

    print(f"After SMOTE, training set sizes for {descriptor_type}:")
    print(f"X_train_res: {X_train_res.shape}, y_train_res: {y_train_res.shape}")

    # ---------------------------
    # 3. Hyperparameter Optimization with Hyperopt
    # ---------------------------

    def objective_catboost(space):
        model = CatBoostClassifier(
            iterations=int(space['iterations']),
            depth=int(space['depth']),
            learning_rate=space['learning_rate'],
            l2_leaf_reg=space['l2_leaf_reg'],
            random_strength=space['random_strength'],
            border_count=int(space['border_count']),
            task_type='CPU',
            random_state=random_state,
            verbose=0
        )

        model.fit(X_train_res, y_train_res, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=0)
        preds = model.predict(X_val)

        # Calculate metrics
        balanced_acc = balanced_accuracy_score(y_val, preds)

        # Hyperopt minimizes the loss
        loss = -balanced_acc

        return {'loss': loss, 'status': STATUS_OK}

    # Define the search space for CatBoost
    space_catboost = {
        'iterations': hp.quniform('iterations', 100, 1000, 50),
        'depth': hp.quniform('depth', 4, 10, 1),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
        'l2_leaf_reg': hp.loguniform('l2_leaf_reg', np.log(1), np.log(10)),
        'random_strength': hp.uniform('random_strength', 0, 1),
        'border_count': hp.quniform('border_count', 32, 255, 1)
    }

    trials_catboost = Trials()
    best_catboost = fmin(
        fn=objective_catboost,
        space=space_catboost,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials_catboost,
        rstate=np.random.default_rng(random_state)
    )

    print(f"Best hyperparameters found for CatBoost ({descriptor_type}): ", best_catboost)

    # Convert hyperparameters to appropriate types
    best_params_catboost = {
        'iterations': int(best_catboost['iterations']),
        'depth': int(best_catboost['depth']),
        'learning_rate': best_catboost['learning_rate'],
        'l2_leaf_reg': best_catboost['l2_leaf_reg'],
        'random_strength': best_catboost['random_strength'],
        'border_count': int(best_catboost['border_count']),
        'task_type': 'CPU',
        'random_state': random_state
    }

    # ---------------------------
    # 4. Train the Best CatBoost Model
    # ---------------------------

    best_model_catboost = CatBoostClassifier(**best_params_catboost)
    best_model_catboost.fit(X_train_res, y_train_res, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=0)

    # ---------------------------
    # 5. Evaluation on Test Set
    # ---------------------------

    y_pred_catboost = best_model_catboost.predict(X_test)
    y_proba_catboost = best_model_catboost.predict_proba(X_test)[:, 1]

    # Calculate metrics
    precision = precision_score(y_test, y_pred_catboost, average='binary')
    recall = recall_score(y_test, y_pred_catboost, average='binary')
    f1 = f1_score(y_test, y_pred_catboost, average='binary')
    balanced_acc = balanced_accuracy_score(y_test, y_pred_catboost)
    auc_score = roc_auc_score(y_test, y_proba_catboost)

    print(f"\nCatBoost Model Performance on Test Set ({descriptor_type}):")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"AUC-ROC:   {auc_score:.4f}")

    # Detailed classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_catboost))

    # ---------------------------
    # 6. Save the Best CatBoost Model
    # ---------------------------

    model_filename_catboost = f'{model_output_path}/best_catboost_model_{descriptor_type}.cbm'
    best_model_catboost.save_model(model_filename_catboost)
    print(f"Best CatBoost model saved to {model_filename_catboost}\n")

    # ---------------------------
    # 7. Compile Performance Metrics
    # ---------------------------

    results_catboost = {
        'Model Type': 'CatBoost',
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Balanced Accuracy': balanced_acc,
        'AUC-ROC': auc_score,
        'Model Path': model_filename_catboost
    }

    return results_catboost


In [ ]:
def run_descriptor_model_screening(
    df: pd.DataFrame,
    target_col: str,
    smiles_col: str='SMILES',
    descriptor_method: str='padel',
    model_output_path: str='models/',
    max_evals: int=10000,
    random_state: int=42
) -> dict:
    """
    Runs the descriptor model screening for a given descriptor method.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing the data.
    target_col : str
        The name of the target column.
    smiles_col : str, optional
        The name of the SMILES column. Defaults to 'SMILES'.
    descriptor_method : str, optional
        The descriptor method to use. One of ['padel', 'maccs', 'fcfp', 'ecfp', 'rdkit'].
    model_output_path : str, optional
        Directory path to save the models and plots. Defaults to 'models/'.
    max_evals : int, optional
        Number of evaluations for hyperparameter optimization. Defaults to 100.
    random_state : int, optional
        Random state for reproducibility. Defaults to 42.

    Returns
    -------
    dict
        A dictionary containing performance metrics and paths.
    """
    print(f"Starting model screening for descriptor method: {descriptor_method}")

    # ---------------------------
    # 1. Extract and Binarize Target
    # ---------------------------
    y = (df['Tendency for the selection of blood samples'] > 0.5).astype(int).values

    # ---------------------------
    # 2. Compute Features X
    # ---------------------------
    try:
        X = compute_descriptors(df, descriptor_type=descriptor_method, smiles_col=smiles_col)
    except ValueError as e:
        print(f"Error computing descriptors: {e}")
        return {}

    # ---------------------------
    # 3. Run Machine Learning Pipeline
    # ---------------------------
    results = run_ml_pipeline(
        X=X,
        y=y,
        descriptor_type=descriptor_method,
        model_output_path=model_output_path,
        max_evals=max_evals,
        random_state=random_state
    )

    return results

In [ ]:
def main():
    """
    Main function to execute the machine learning pipeline for each descriptor type.
    """

    # Define descriptor methods you want to test
    descriptor_methods = ['padel', 'maccs', 'fcfp', 'ecfp', 'rdkit']

    # Dictionary to store the results for each descriptor method
    all_results = {}

    for method in descriptor_methods:
        results = run_descriptor_model_screening(
            df=df,
            target_col='Tendency for the selection of blood samples',
            smiles_col='SMILES',
            descriptor_method=method,
            model_output_path='models/',       # Ensure this directory exists or is created
            max_evals=2000,                     # Adjust based on computational resources
            random_state=42
        )
        all_results[method] = results

    # ---------------------------
    # 4. Save All Results to a CSV
    # ---------------------------

    results_df = pd.DataFrame(all_results).T  # Transpose for better readability
    results_csv_path = 'models/all_descriptor_results.csv'
    results_df.to_csv(results_csv_path)
    print(f"All results saved to {results_csv_path}")

    # ---------------------------
    # 5. Display Summary
    # ---------------------------

    print("\nSummary of All Descriptor Methods:")
    print(results_df)

if __name__ == "__main__":
    main()

Starting model screening for descriptor method: padel
Dataset sizes for padel:
Training set: X_train: (321, 664), y_train: (321,)
Validation set: X_val: (71, 664), y_val: (71,)
Test set: X_test: (144, 664), y_test: (144,)
After SMOTE, training set sizes for padel:
X_train_res: (396, 664), y_train_res: (396,)
100%|██████████| 2000/2000 [4:31:35<00:00,  8.15s/trial, best loss: -0.9545454545454546]
Best hyperparameters found for CatBoost (padel):  {'border_count': 218.0, 'depth': 7.0, 'iterations': 800.0, 'l2_leaf_reg': 5.024063556864772, 'learning_rate': 0.28433220907329426, 'random_strength': 0.4864672634101255}

CatBoost Model Performance on Test Set (padel):
Precision: 0.9310
Recall:    0.9205
F1-Score:  0.9257
Balanced Accuracy: 0.9067
AUC-ROC:   0.9750

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.89      0.88        56
           1       0.93      0.92      0.93        88

    accuracy                           0.91   